In [120]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]")\
          .appName("sunshine_v2")\
          .getOrCreate()

In [137]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.ml.feature import *

In [122]:
df = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("final_feature.csv")

df.show(vertical=True)

-RECORD 0---------------------------------------------------------------------------
 features_smile.msisdn                                       | 22502251164          
 features_smile.nombre_profil                                | NULL                 
 features_smile.nombre_fois_profil_change                    | NULL                 
 features_smile.profil                                       | NULL                 
 features_smile.nombre_souscription                          | NULL                 
 features_smile.nombre_pass_achete                           | NULL                 
 features_smile.nombre_sms_sortants                          | NULL                 
 features_smile.nombre_correspondant_sms_sortant             | NULL                 
 features_smile.nombre_appels_sortants                       | NULL                 
 features_smile.nombre_correspondant_appels_sortant          | NULL                 
 features_smile.nombre_correspondant_distinct_appels_sortant | NU

In [123]:
df.printSchema()

root
 |-- features_smile.msisdn: string (nullable = true)
 |-- features_smile.nombre_profil: string (nullable = true)
 |-- features_smile.nombre_fois_profil_change: string (nullable = true)
 |-- features_smile.profil: string (nullable = true)
 |-- features_smile.nombre_souscription: string (nullable = true)
 |-- features_smile.nombre_pass_achete: string (nullable = true)
 |-- features_smile.nombre_sms_sortants: string (nullable = true)
 |-- features_smile.nombre_correspondant_sms_sortant: string (nullable = true)
 |-- features_smile.nombre_appels_sortants: string (nullable = true)
 |-- features_smile.nombre_correspondant_appels_sortant: string (nullable = true)
 |-- features_smile.nombre_correspondant_distinct_appels_sortant: string (nullable = true)
 |-- features_smile.duree_totale_appel_sortant: string (nullable = true)
 |-- features_smile.duree_moyenne_appel_sortant: string (nullable = true)
 |-- features_smile.nombre_sms_entrants: string (nullable = true)
 |-- features_smile.nombre

In [124]:
df1 = df.drop("features_smile.nombre_profil", "features_smile.nombre_fois_profil_change", "features_smile.profil", "features_smile.nombre_souscription", "features_smile.nombre_pass_achete")

In [125]:
df1.columns

['features_smile.msisdn',
 'features_smile.nombre_sms_sortants',
 'features_smile.nombre_correspondant_sms_sortant',
 'features_smile.nombre_appels_sortants',
 'features_smile.nombre_correspondant_appels_sortant',
 'features_smile.nombre_correspondant_distinct_appels_sortant',
 'features_smile.duree_totale_appel_sortant',
 'features_smile.duree_moyenne_appel_sortant',
 'features_smile.nombre_sms_entrants',
 'features_smile.nombre_correspondant_sms_entrant',
 'features_smile.nombre_appel_entrants',
 'features_smile.nombre_correspondant_appel_entrant',
 'features_smile.nombre_correspondant_appel_distinct_entrant',
 'features_smile.duree_totale_appel_entrant',
 'features_smile.duree_moyenne_appel_entrant',
 'features_smile.nombre_sms',
 'features_smile.nombre_appels',
 'features_smile.nombre_correspondants_sms',
 'features_smile.nombre_correspondants',
 'features_smile.nombre_correspondants_distincts',
 'features_smile.nombre_correspondants_distincts_appels',
 'features_smile.nombre_corre

In [126]:
liste =['features_smile.msisdn',
 'features_smile.nombre_sms_sortants',
 'features_smile.nombre_correspondant_sms_sortant',
 'features_smile.nombre_appels_sortants',
 'features_smile.nombre_correspondant_appels_sortant',
 'features_smile.nombre_correspondant_distinct_appels_sortant',
 'features_smile.duree_totale_appel_sortant',
 'features_smile.duree_moyenne_appel_sortant',
 'features_smile.nombre_sms_entrants',
 'features_smile.nombre_correspondant_sms_entrant',
 'features_smile.nombre_appel_entrants',
 'features_smile.nombre_correspondant_appel_entrant',
 'features_smile.nombre_correspondant_appel_distinct_entrant',
 'features_smile.duree_totale_appel_entrant',
 'features_smile.duree_moyenne_appel_entrant',
 'features_smile.nombre_sms',
 'features_smile.nombre_appels',
 'features_smile.nombre_correspondants_sms',
 'features_smile.nombre_correspondants',
 'features_smile.nombre_correspondants_distincts',
 'features_smile.nombre_correspondants_distincts_appels',
 'features_smile.nombre_correspondants_appels',
 'features_smile.duree_totale_appels',
 'features_smile.duree_moyenne_appels',
 'features_smile.taux_inities_sms',
 'features_smile.nombre_appels_onnet',
 'features_smile.taux_appels_inities_onnet',
 'features_smile.nombre_appels_offnet',
 'features_smile.taux_appels_inities_offnet',
 'features_smile.nombre_appels_inter',
 'features_smile.taux_appels_inities_inter',
 'features_smile.nombre_appels_complice',
 'features_smile.taux_appels_inities_complice',
 'features_smile.nbre_imsi',
 'features_smile.nbre_distinct_imei',
 'features_smile.ratio_nombre_imsi_imei',
 'features_smile.nombre_appels_nuit',
 'features_smile.nombre_appels_nuit_distinct',
 'features_smile.temps_moyen_evenement',
 'features_smile.variance_evenement',
 'features_smile.temps_moyen_appel',
 'features_smile.variance_appel',
 'features_smile.temps_moyen_sms',
 'features_smile.variance_sms',
 'features_smile.totaldureeusedata',
 'features_smile.nbre_sim',
 'features_smile.supplier_name',
 'features_smile.nombre_cal_onnet',
 'features_smile.nbre_moy_cal_onnet',
 'features_smile.duree_moy_cal_onnet',
 'features_smile.duree_cal_onnet',
 'features_smile.nbre_moy_cal_offnet',
 'features_smile.nombre_cal_offnet',
 'features_smile.duree_moy_cal_offnet',
 'features_smile.duree_cal_offnet',
 'features_smile.duree_mean_call_out',
 'features_smile.ratio_corresp_out_in',
 'features_smile.ratio_call_out_in',
 'features_smile.numb_inter_correp_out_distinct',
 'features_smile.numb_inter_correp_out',
 'features_smile.time_call_out',
 'features_smile.numb_call_inter_out',
 'features_smile.numb_inter_correp_in_distinct',
 'features_smile.numb_inter_correp_in',
 'features_smile.time_call_in',
 'features_smile.numb_call_inter_in',
 'features_smile.numb_inter_corresp_distinct',
 'features_smile.numb_inter_correp',
 'features_smile.numb_call_inter']

In [40]:
for cl in liste:
  df1 = df1.withColumn(cl,col(cl).cast(DoubleType)

SyntaxError: unexpected EOF while parsing (<ipython-input-40-657168aa80d8>, line 2)

In [60]:
df1.select("features_smile.nombre_sms_sortants").show()

AnalysisException: cannot resolve '`features_smile.nombre_sms_sortants`' given input columns: [features_smile.duree_cal_offnet, features_smile.duree_cal_onnet, features_smile.duree_mean_call_out, features_smile.duree_moy_cal_offnet, features_smile.duree_moy_cal_onnet, features_smile.duree_moyenne_appel_entrant, features_smile.duree_moyenne_appel_sortant, features_smile.duree_moyenne_appels, features_smile.duree_totale_appel_entrant, features_smile.duree_totale_appel_sortant, features_smile.duree_totale_appels, features_smile.msisdn, features_smile.nbre_distinct_imei, features_smile.nbre_imsi, features_smile.nbre_moy_cal_offnet, features_smile.nbre_moy_cal_onnet, features_smile.nbre_sim, features_smile.nombre_appel_entrants, features_smile.nombre_appels, features_smile.nombre_appels_complice, features_smile.nombre_appels_inter, features_smile.nombre_appels_nuit, features_smile.nombre_appels_nuit_distinct, features_smile.nombre_appels_offnet, features_smile.nombre_appels_onnet, features_

In [48]:
df1.withColumn("features_smile.numb_call_inter", col("features_smile.numb_call_inter").cast(DoubleType()))

AnalysisException: cannot resolve '`features_smile.numb_call_inter`' given input columns: [features_smile.duree_cal_offnet, features_smile.duree_cal_onnet, features_smile.duree_mean_call_out, features_smile.duree_moy_cal_offnet, features_smile.duree_moy_cal_onnet, features_smile.duree_moyenne_appel_entrant, features_smile.duree_moyenne_appel_sortant, features_smile.duree_moyenne_appels, features_smile.duree_totale_appel_entrant, features_smile.duree_totale_appel_sortant, features_smile.duree_totale_appels, features_smile.msisdn, features_smile.nbre_distinct_imei, features_smile.nbre_imsi, features_smile.nbre_moy_cal_offnet, features_smile.nbre_moy_cal_onnet, features_smile.nbre_sim, features_smile.nombre_appel_entrants, features_smile.nombre_appels, features_smile.nombre_appels_complice, features_smile.nombre_appels_inter, features_smile.nombre_appels_nuit, features_smile.nombre_appels_nuit_distinct, features_smile.nombre_appels_offnet, features_smile.nombre_appels_onnet, features_smil

In [50]:
df1.withColumn("features_smile.numb_call_inter", df1["features_smile.numb_call_inter"].cast("double"))

AnalysisException: Cannot resolve column name "features_smile.numb_call_inter" among (features_smile.msisdn, features_smile.nombre_sms_sortants, features_smile.nombre_correspondant_sms_sortant, features_smile.nombre_appels_sortants, features_smile.nombre_correspondant_appels_sortant, features_smile.nombre_correspondant_distinct_appels_sortant, features_smile.duree_totale_appel_sortant, features_smile.duree_moyenne_appel_sortant, features_smile.nombre_sms_entrants, features_smile.nombre_correspondant_sms_entrant, features_smile.nombre_appel_entrants, features_smile.nombre_correspondant_appel_entrant, features_smile.nombre_correspondant_appel_distinct_entrant, features_smile.duree_totale_appel_entrant, features_smile.duree_moyenne_appel_entrant, features_smile.nombre_sms, features_smile.nombre_appels, features_smile.nombre_correspondants_sms, features_smile.nombre_correspondants, features_smile.nombre_correspondants_distincts, features_smile.nombre_correspondants_distincts_appels, featur

In [59]:
df1.select("features_smile.duree_cal_onnet").show()

AnalysisException: cannot resolve '`features_smile.duree_cal_onnet`' given input columns: [features_smile.duree_cal_offnet, features_smile.duree_cal_onnet, features_smile.duree_mean_call_out, features_smile.duree_moy_cal_offnet, features_smile.duree_moy_cal_onnet, features_smile.duree_moyenne_appel_entrant, features_smile.duree_moyenne_appel_sortant, features_smile.duree_moyenne_appels, features_smile.duree_totale_appel_entrant, features_smile.duree_totale_appel_sortant, features_smile.duree_totale_appels, features_smile.msisdn, features_smile.nbre_distinct_imei, features_smile.nbre_imsi, features_smile.nbre_moy_cal_offnet, features_smile.nbre_moy_cal_onnet, features_smile.nbre_sim, features_smile.nombre_appel_entrants, features_smile.nombre_appels, features_smile.nombre_appels_complice, features_smile.nombre_appels_inter, features_smile.nombre_appels_nuit, features_smile.nombre_appels_nuit_distinct, features_smile.nombre_appels_offnet, features_smile.nombre_appels_onnet, features_smil

In [54]:
toDoublefunc = UserDefinedFunction(lambda x: float(x),DoubleType())
df1.withColumn("features_smile.numb_call_inter",toDoublefunc(df1["features_smile.numb_call_inter"]))

AnalysisException: Cannot resolve column name "features_smile.numb_call_inter" among (features_smile.msisdn, features_smile.nombre_sms_sortants, features_smile.nombre_correspondant_sms_sortant, features_smile.nombre_appels_sortants, features_smile.nombre_correspondant_appels_sortant, features_smile.nombre_correspondant_distinct_appels_sortant, features_smile.duree_totale_appel_sortant, features_smile.duree_moyenne_appel_sortant, features_smile.nombre_sms_entrants, features_smile.nombre_correspondant_sms_entrant, features_smile.nombre_appel_entrants, features_smile.nombre_correspondant_appel_entrant, features_smile.nombre_correspondant_appel_distinct_entrant, features_smile.duree_totale_appel_entrant, features_smile.duree_moyenne_appel_entrant, features_smile.nombre_sms, features_smile.nombre_appels, features_smile.nombre_correspondants_sms, features_smile.nombre_correspondants, features_smile.nombre_correspondants_distincts, features_smile.nombre_correspondants_distincts_appels, featur

In [11]:
df1 = df1.fillna(0)

In [14]:
# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(df1)

# Make predictions
predictions = model.transform(df1)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

IllegalArgumentException: features does not exist. Available: features_smile.msisdn, features_smile.nombre_sms_sortants, features_smile.nombre_correspondant_sms_sortant, features_smile.nombre_appels_sortants, features_smile.nombre_correspondant_appels_sortant, features_smile.nombre_correspondant_distinct_appels_sortant, features_smile.duree_totale_appel_sortant, features_smile.duree_moyenne_appel_sortant, features_smile.nombre_sms_entrants, features_smile.nombre_correspondant_sms_entrant, features_smile.nombre_appel_entrants, features_smile.nombre_correspondant_appel_entrant, features_smile.nombre_correspondant_appel_distinct_entrant, features_smile.duree_totale_appel_entrant, features_smile.duree_moyenne_appel_entrant, features_smile.nombre_sms, features_smile.nombre_appels, features_smile.nombre_correspondants_sms, features_smile.nombre_correspondants, features_smile.nombre_correspondants_distincts, features_smile.nombre_correspondants_distincts_appels, features_smile.nombre_correspo

In [29]:
null + 0

NameError: name 'null' is not defined

In [61]:
achat = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       csv("features_pass_souscription_profil.csv")

achat.show(vertical=True)

-RECORD 0---------------------------------------------------------------------------
 features_pass_souscription_profil.msisdn                    | 22507004594          
 features_pass_souscription_profil.nombre_profil             | 1                    
 features_pass_souscription_profil.nombre_fois_profil_change | 0                    
 features_pass_souscription_profil.profil                    | Basic Price Plan ... 
 features_pass_souscription_profil.nombre_souscription       | 5                    
 features_pass_souscription_profil.nombre_pass_achete        | 12                   
-RECORD 1---------------------------------------------------------------------------
 features_pass_souscription_profil.msisdn                    | 22507007581          
 features_pass_souscription_profil.nombre_profil             | NULL                 
 features_pass_souscription_profil.nombre_fois_profil_change | NULL                 
 features_pass_souscription_profil.profil                    | NU

In [72]:
df2 = df1.join(achat, df1["`features_smile.msisdn`"] == achat["`features_pass_souscription_profil.msisdn`"], "inner").drop("features_pass_souscription_profil.msisdn")

In [73]:
df2.write.parquet("final.parquet")

# New analysis

In [ ]:
df = spark.read.\
       option("header", "true").\
       option("delimiter", ";").\
       parquet("final.parquet")

df.show(vertical=True)

In [ ]:
df.printSchema()

In [ ]:
df1 = df
df1.show(vertical = True)

In [ ]:
df1 = df1.na.fill(0)

In [ ]:
df1.show(vertical = True)

In [ ]:
liste = [
 'features_smile_nombre_sms_sortants',
 'features_smile_nombre_correspondant_sms_sortant',
 'features_smile_nombre_appels_sortants',
 'features_smile_nombre_correspondant_appels_sortant',
 'features_smile_nombre_correspondant_distinct_appels_sortant',
 'features_smile_duree_totale_appel_sortant',
 'features_smile_duree_moyenne_appel_sortant',
 'features_smile_nombre_sms_entrants',
 'features_smile_nombre_correspondant_sms_entrant',
 'features_smile_nombre_appel_entrants',
 'features_smile_nombre_correspondant_appel_entrant',
 'features_smile_nombre_correspondant_appel_distinct_entrant',
 'features_smile_duree_totale_appel_entrant',
 'features_smile_duree_moyenne_appel_entrant',
 'features_smile_nombre_sms',
 'features_smile_nombre_appels',
 'features_smile_nombre_correspondants_sms',
 'features_smile_nombre_correspondants',
 'features_smile_nombre_correspondants_distincts',
 'features_smile_nombre_correspondants_distincts_appels',
 'features_smile_nombre_correspondants_appels',
 'features_smile_duree_totale_appels',
 'features_smile_duree_moyenne_appels',
 'features_smile_taux_inities_sms',
 'features_smile_nombre_appels_onnet',
 'features_smile_taux_appels_inities_onnet',
 'features_smile_nombre_appels_offnet',
 'features_smile_taux_appels_inities_offnet',
 'features_smile_nombre_appels_inter',
 'features_smile_taux_appels_inities_inter',
 'features_smile_nombre_appels_complice',
 'features_smile_taux_appels_inities_complice',
 'features_smile_nbre_imsi',
 'features_smile_nbre_distinct_imei',
 'features_smile_ratio_nombre_imsi_imei',
 'features_smile_nombre_appels_nuit',
 'features_smile_nombre_appels_nuit_distinct',
 'features_smile_temps_moyen_evenement',
 'features_smile_variance_evenement',
 'features_smile_temps_moyen_appel',
 'features_smile_variance_appel',
 'features_smile_temps_moyen_sms',
 'features_smile_variance_sms',
 'features_smile_totaldureeusedata',
 'features_smile_nbre_sim',
 'features_smile_supplier_name',
 'features_smile_nombre_cal_onnet',
 'features_smile_nbre_moy_cal_onnet',
 'features_smile_duree_moy_cal_onnet',
 'features_smile_duree_cal_onnet',
 'features_smile_nbre_moy_cal_offnet',
 'features_smile_nombre_cal_offnet',
 'features_smile_duree_moy_cal_offnet',
 'features_smile_duree_cal_offnet',
 'features_smile_duree_mean_call_out',
 'features_smile_ratio_corresp_out_in',
 'features_smile_ratio_call_out_in',
 'features_smile_numb_inter_correp_out_distinct',
 'features_smile_numb_inter_correp_out',
 'features_smile_time_call_out',
 'features_smile_numb_call_inter_out',
 'features_smile_numb_inter_correp_in_distinct',
 'features_smile_numb_inter_correp_in',
 'features_smile_time_call_in',
 'features_smile_numb_call_inter_in',
 'features_smile_numb_inter_corresp_distinct',
 'features_smile_numb_inter_correp',
 'features_smile_numb_call_inter',
 'features_pass_souscription_profil_nombre_profil',
 'features_pass_souscription_profil_nombre_fois_profil_change',
 'features_pass_souscription_profil_nombre_souscription',
 'features_pass_souscription_profil_nombre_pass_achete']

In [ ]:
df1 = df1.toDF(*(c.replace('.', '_') for c in df1.columns))

In [ ]:
for cl in liste:
    df1 = df1.withColumn(cl, col(cl).cast(DoubleType()))

In [ ]:
df1.printSchema()

In [ ]:
#features_pass_souscription_profil.profil_indexer = StringIndexer(inputCol="features_pass_souscription_profil_profil", outputCol='features_pass_souscription_profil_profilIndex')

#features_pass_souscription_profil.profil_vector = OneHotEncoder(inputCol="features_pass_souscription_profil_profilIndex", outputCol="features_pass_souscription_profil_profil_vec")

assemble=VectorAssembler(inputCols=[
 'features_smile_nombre_sms_sortants',
 'features_smile_nombre_correspondant_sms_sortant',
 'features_smile_nombre_appels_sortants',
 'features_smile_nombre_correspondant_appels_sortant',
 'features_smile_nombre_correspondant_distinct_appels_sortant',
 'features_smile_duree_totale_appel_sortant',
 'features_smile_duree_moyenne_appel_sortant',
 'features_smile_nombre_sms_entrants',
 'features_smile_nombre_correspondant_sms_entrant',
 'features_smile_nombre_appel_entrants',
 'features_smile_nombre_correspondant_appel_entrant',
 'features_smile_nombre_correspondant_appel_distinct_entrant',
 'features_smile_duree_totale_appel_entrant',
 'features_smile_duree_moyenne_appel_entrant',
 'features_smile_nombre_sms',
 'features_smile_nombre_appels',
 'features_smile_nombre_correspondants_sms',
 'features_smile_nombre_correspondants',
 'features_smile_nombre_correspondants_distincts',
 'features_smile_nombre_correspondants_distincts_appels',
 'features_smile_nombre_correspondants_appels',
 'features_smile_duree_totale_appels',
 'features_smile_duree_moyenne_appels',
 'features_smile_taux_inities_sms',
 'features_smile_nombre_appels_onnet',
 'features_smile_taux_appels_inities_onnet',
 'features_smile_nombre_appels_offnet',
 'features_smile_taux_appels_inities_offnet',
 'features_smile_nombre_appels_inter',
 'features_smile_taux_appels_inities_inter',
 'features_smile_nombre_appels_complice',
 'features_smile_taux_appels_inities_complice',
 'features_smile_nbre_imsi',
 'features_smile_nbre_distinct_imei',
 'features_smile_ratio_nombre_imsi_imei',
 'features_smile_nombre_appels_nuit',
 'features_smile_nombre_appels_nuit_distinct',
 'features_smile_temps_moyen_evenement',
 'features_smile_variance_evenement',
 'features_smile_temps_moyen_appel',
 'features_smile_variance_appel',
 'features_smile_temps_moyen_sms',
 'features_smile_variance_sms',
 'features_smile_totaldureeusedata',
 'features_smile_nbre_sim',
 'features_smile_supplier_name',
 'features_smile_nombre_cal_onnet',
 'features_smile_nbre_moy_cal_onnet',
 'features_smile_duree_moy_cal_onnet',
 'features_smile_duree_cal_onnet',
 'features_smile_nbre_moy_cal_offnet',
 'features_smile_nombre_cal_offnet',
 'features_smile_duree_moy_cal_offnet',
 'features_smile_duree_cal_offnet',
 'features_smile_duree_mean_call_out',
 'features_smile_ratio_corresp_out_in',
 'features_smile_ratio_call_out_in',
 'features_smile_numb_inter_correp_out_distinct',
 'features_smile_numb_inter_correp_out',
 'features_smile_time_call_out',
 'features_smile_numb_call_inter_out',
 'features_smile_numb_inter_correp_in_distinct',
 'features_smile_numb_inter_correp_in',
 'features_smile_time_call_in',
 'features_smile_numb_call_inter_in',
 'features_smile_numb_inter_corresp_distinct',
 'features_smile_numb_inter_correp',
 'features_smile_numb_call_inter',
 'features_pass_souscription_profil_nombre_profil',
 'features_pass_souscription_profil_nombre_fois_profil_change',
 'features_pass_souscription_profil_nombre_souscription',
 'features_pass_souscription_profil_nombre_pass_achete'], outputCol='features')


#pipeline = Pipeline(stages=[assemble])

In [ ]:
assembled_data = assemble.transform(df1)

In [ ]:
scale = StandardScaler(inputCol='features',outputCol='standardized')
data_scale = scale.fit(assembled_data)
data_scale_output = data_scale.transform(assembled_data)
data_scale_output.show(2)

In [197]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')


KMeans_algo = KMeans(featuresCol='standardized', k=2)
    
KMeans_fit = KMeans_algo.fit(data_scale_output)
    
output = KMeans_fit.transform(data_scale_output)

score = evaluator.evaluate(output)

silhouette_score.append(score)

print("Silhouette Score:",score)

KeyboardInterrupt: 

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')


KMeans_algo = KMeans(featuresCol='standardized', k=3)
    
KMeans_fit = KMeans_algo.fit(data_scale_output)
    
output = KMeans_fit.transform(data_scale_output)

score = evaluator.evaluate(output)

silhouette_score.append(score)

print("Silhouette Score:",score)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')


KMeans_algo = KMeans(featuresCol='standardized', k=4)
    
KMeans_fit = KMeans_algo.fit(data_scale_output)
    
output = KMeans_fit.transform(data_scale_output)

score = evaluator.evaluate(output)

silhouette_score.append(score)

print("Silhouette Score:",score)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')


KMeans_algo = KMeans(featuresCol='standardized', k=5)
    
KMeans_fit = KMeans_algo.fit(data_scale_output)
    
output = KMeans_fit.transform(data_scale_output)

score = evaluator.evaluate(output)

silhouette_score.append(score)

print("Silhouette Score:",score)

In [195]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
for i in range(2,10):
    
    KMeans_algo = KMeans(featuresCol='standardized', k=i)
    
    KMeans_fit = KMeans_algo.fit(data_scale_output)
    
    output = KMeans_fit.transform(data_scale_output)
    
    score = evaluator.evaluate(output)
    
    silhouette_score.append(score)
    
    print("Silhouette Score:",score)

KeyboardInterrupt: 

Visualizing the silhouette score. Note previous versions of the K Means had computeScore() that calculated the sum of intracluster distance but got deprecated in Spark 3.0.0. Silhouette Score using ClusteringEvaluator() measures how close each point in one cluster is to points in the neighboring clusters thus helping in figuring out clusters that are compact and well-spaced out.

In [ ]:
#Visualizing the silhouette scores in a plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),silhouette_score)
ax.set_xlabel(‘k’)
ax.set_ylabel(‘cost’)

In [ ]:
nombre de lignes , nombre de colonnes, valeurs manquantes, outliers(international) , corrélation .